# **Microsoft Malware Prediction **
The purpose of this kernel is to investigate and predict the likelihood of Windows machines getting Marlware infections based on the properties of the machine.

In the train dataset, `HasDetections` indicates whether a machine with a particular `MachineIdentifier` is infected by Malware and in the test dataset `HasDetections` is the target variable that we aim to predict.

# Strategy

1. Data Discovery - size, dimensions, attributes, datatypes, etc.
2. Categorical Exploratory Data Analysis 
3. Numerical Exploratory Data Analysis 

In [ ]:
# Import the required libraries

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
sns.set_style(style = 'whitegrid')

## Import & Investigate the input files (train.csv,test.csv)

Important Note: the input files are too big to load all at once using *pd.lead_csv()*, therefore I only loaded 100 records of the train dataset into a temporarty DataFrame to get a sense of what's inside the file. 

In [ ]:
## load the test dataset into a temp dataframe
tmp_train_df = pd.read_csv('../input/train.csv', nrows=100)
no_of_rows,no_of_cols = tmp_train_df.shape
print('The temp train dataset has: ', no_of_rows,'Rows', '&',no_of_cols,'Columns' )

In [ ]:
## load the test dataset into a temp dataframe
tmp_test_df = pd.read_csv('../input/test.csv', nrows=100)
no_rows,no_cols = tmp_test_df.shape
print('The temp test dataset has: ', no_rows,'Rows', '&',no_cols,'Columns' )

In [ ]:
# List the 83 columns and check their datatypes
tmp_train_df.info()

In [ ]:
# List the 82 columns and check their datatypes
tmp_test_df.info()

## Observations:

1. There are three datatypes in the train dataset: 
 * float64: 23 columns
 * int64: 32 columns
 * object: 28 columns

2. There are three datatypes in the test dataset:
 * float64: 26 columns
 * int64: 28 columns
 * object: 28 columns

3. memory usage for loading 100 rows: 64.9+ KM

# Loading the entire dataset into a DataFrame 
To load the entire dataset, I decided to adopt the approach of [this useful kernel](https://www.kaggle.com/theoviel/load-the-totality-of-the-data) by  [Theo Viel](https://www.kaggle.com/theoviel) that suggests changing the datatypes will seemingly reduce the memory usage and allow the data to be loaded into memory fully. 

In [ ]:
dtypes = {
        'MachineIdentifier':                                    'category'
        ,'ProductName':                                          'category'
        ,'EngineVersion':                                        'category'
        ,'AppVersion':                                           'category'
        ,'AvSigVersion':                                         'category'
        ,'IsBeta':                                               'int8'
        ,'RtpStateBitfield':                                     'float16'
        ,'IsSxsPassiveMode':                                     'int8'
        ,'DefaultBrowsersIdentifier':                            'float16'
        ,'AVProductStatesIdentifier':                            'float32'
        ,'AVProductsInstalled':                                  'float16'
        ,'AVProductsEnabled':                                    'float16'
        ,'HasTpm':                                               'int8'
        ,'CountryIdentifier':                                    'int16'
        ,'CityIdentifier':                                       'float32'
        ,'OrganizationIdentifier':                               'float16'
        ,'GeoNameIdentifier':                                    'float16'
        ,'LocaleEnglishNameIdentifier':                          'int8'
        ,'Platform':                                             'category'
        ,'Processor':                                            'category'
        ,'OsVer':                                                'category'
        ,'OsBuild':                                              'int16'
        ,'OsSuite':                                              'int16'
        ,'OsPlatformSubRelease':                                 'category'
        ,'OsBuildLab':                                           'category'
        ,'SkuEdition':                                           'category'
        ,'IsProtected':                                          'float16'
        ,'AutoSampleOptIn':                                      'int8'
        ,'PuaMode':                                              'category'
        ,'SMode':                                                'float16'
        ,'IeVerIdentifier':                                      'float16'
        ,'SmartScreen':                                          'category'
        ,'Firewall':                                             'float16'
        ,'UacLuaenable':                                         'float32'
        ,'Census_MDC2FormFactor':                                'category'
        ,'Census_DeviceFamily':                                  'category'
        ,'Census_OEMNameIdentifier':                             'float16'
        ,'Census_OEMModelIdentifier':                            'float32'
        ,'Census_ProcessorCoreCount':                            'float16'
        ,'Census_ProcessorManufacturerIdentifier':               'float16'
        ,'Census_ProcessorModelIdentifier':                      'float16'
        ,'Census_ProcessorClass':                                'category'
        ,'Census_PrimaryDiskTotalCapacity':                      'float32'
        ,'Census_PrimaryDiskTypeName':                           'category'
        ,'Census_SystemVolumeTotalCapacity':                     'float32'
        ,'Census_HasOpticalDiskDrive':                           'int8'
        ,'Census_TotalPhysicalRAM':                              'float32'
        ,'Census_ChassisTypeName':                               'category'
        ,'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16'
        ,'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16'
        ,'Census_InternalPrimaryDisplayResolutionVertical':      'float16'
        ,'Census_PowerPlatformRoleName':                         'category'
        ,'Census_InternalBatteryType':                           'category'
        ,'Census_InternalBatteryNumberOfCharges':                'float32'
        ,'Census_OSVersion':                                     'category'
        ,'Census_OSArchitecture':                                'category'
        ,'Census_OSBranch':                                      'category'
        ,'Census_OSBuildNumber':                                 'int16'
        ,'Census_OSBuildRevision':                               'int32'
        ,'Census_OSEdition':                                     'category'
        ,'Census_OSSkuName':                                     'category'
        ,'Census_OSInstallTypeName':                             'category'
        ,'Census_OSInstallLanguageIdentifier':                   'float16'
        ,'Census_OSUILocaleIdentifier':                          'int16'
        ,'Census_OSWUAutoUpdateOptionsName':                     'category'
        ,'Census_IsPortableOperatingSystem':                     'int8'
        ,'Census_GenuineStateName':                              'category'
        ,'Census_ActivationChannel':                             'category'
        ,'Census_IsFlightingInternal':                           'float16'
        ,'Census_IsFlightsDisabled':                             'float16'
        ,'Census_FlightRing':                                    'category'
        ,'Census_ThresholdOptIn':                                'float16'
        ,'Census_FirmwareManufacturerIdentifier':                'float16'
        ,'Census_FirmwareVersionIdentifier':                     'float32'
        ,'Census_IsSecureBootEnabled':                           'int8'
        ,'Census_IsWIMBootEnabled':                              'float16'
        ,'Census_IsVirtualDevice':                               'float16'
        ,'Census_IsTouchEnabled':                                'int8'
        ,'Census_IsPenCapable':                                  'int8'
        ,'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16'
        ,'Wdft_IsGamer':                                         'float16'
        ,'Wdft_RegionIdentifier':                                'float16'
        ,'HasDetections':                                        'int8'
        }

In [ ]:
# Load the entire train dataset into a dataframe
%time train_df = pd.read_csv('../input/train.csv', dtype=dtypes)

In [ ]:
# Check out the new datatypes in the dataframe
train_df.info()

In [ ]:
# Repeat the process for loading the entire test dataset into a dataframe
%time test_df = pd.read_csv('../input/test.csv', dtype=dtypes)

# Summarize the train dataset

In [ ]:
# Get the summary statistics of the train variables
train_df.describe()

# EDA for Categorical Features

Here,I perform an EDA on categorical variables using the method discussed in [this interesting kernel](https://www.kaggle.com/nextbigwhat/eda-for-categorical-variables-part-2) by [Samarth Agrawal](https://www.kaggle.com/nextbigwhat).

In [ ]:
# Create a dataframe of categorical features only
ds_cat = train_df.select_dtypes(include = 'category').copy()

For each feature, get the distinct values, number of values including nulls,number of non-null values and percentage of null values. 

In [ ]:
# create an empty dataframe for the summary statistics of categorical data
ds_cat_stats = pd.DataFrame(columns = ['column', 'values', 'values_count_incna', 'values_count_nona', 
                                       'num_miss', 'pct_miss'])
tmp = pd.DataFrame()

for c in ds_cat.columns:
    tmp['column'] = [c]
    tmp['values'] = [ds_cat[c].unique()]
    tmp['values_count_incna'] = len(list(ds_cat[c].unique()))
    tmp['values_count_nona'] = int(ds_cat[c].nunique())
    tmp['num_miss'] = ds_cat[c].isnull().sum()
    tmp['pct_miss'] = (ds_cat[c].isnull().sum()/ len(ds_cat)).round(3)*100
    ds_cat_stats = ds_cat_stats.append(tmp)
    
ds_cat_stats
